In [1]:
import pandas as pd
import numpy as np

In [2]:
data = pd.read_excel('Documents/DivarTask/Summer Camp Task Data.xlsx')

In [3]:
def tok(str):
    str = str.replace('[','')
    str = str.replace(']','')
    tokens = str.split(',')
    return tokens

In [4]:
filtered = data.loc[data['post_page_offset'] == 0].reset_index()
numb_ads = filtered['tokens'].apply(tok).apply(len)

In [5]:
numb_dark = len(numb_ads[numb_ads < 10])
numb_dark

1190

In [6]:
numb_query = data.source_event_id.unique().size
numb_query

14296

In [7]:
dark_query_percent = numb_dark/numb_query
dark_query_percent

0.08324006715165082

In [8]:
load_post_query = set(data[data['action'] == 'load_post_page']['source_event_id'].unique())
click_post_query = set(data[data['action'] == 'click_post']['source_event_id'].unique())

In [9]:
unclicked = list(load_post_query-click_post_query)
numb_unclicked = len(unclicked)
numb_unclicked

3476

In [10]:
query_bounce_rate = numb_unclicked/numb_query
query_bounce_rate

0.24314493564633463

In [46]:
copy_data = data.copy()

In [47]:
copy_data.loc[copy_data['action']=='click_post', 'numb_clicks'] = 1
copy_data.loc[copy_data['action']=='load_post_page', 'numb_clicks'] = 0

In [48]:
copy_data['numb_ads'] = copy_data.loc[copy_data['action']=='load_post_page']['tokens'].apply(tok).apply(len)
copy_data.loc[copy_data['action']=='click_post', 'numb_ads'] = 0

In [79]:
query_data = copy_data.groupby('source_event_id', as_index=False)[['numb_ads', 'numb_clicks']].agg(np.sum)
query_data['clicks_rate'] = query_data['numb_clicks']/query_data['numb_ads']
query_data.loc[query_data['numb_ads']==0, 'clicks_rate'] = 0
query_data.aggregate(np.mean)

numb_ads       56.213556
numb_clicks     5.301903
clicks_rate     0.062037
dtype: float64

In [77]:
copy_data['created_at'] = copy_data['created_at'].apply(pd.Timestamp, unit='ms')

In [80]:
temp_data = copy_data.loc[copy_data['action']=='click_post'].groupby('source_event_id', as_index=False)['created_at'].idxmin()
click = copy_data.loc[temp_data['created_at'],:][['source_event_id','post_index_in_post_list']].reset_index()
del click['index']
click = click.rename(columns={'post_index_in_post_list':'first_click_index'})

In [81]:
query_data = pd.merge(left = query_data, right = click, how='left')
query_data

,source_event_id,numb_ads,numb_clicks,clicks_rate,first_click_index
0,00049ef3-c027-4f55-869c-46711f0ea0ad,24.0,0.0,0.00000,NaN
1,00069dda-4175-4bb0-8ef0-c0e1031d4da6,24.0,0.0,0.00000,NaN
2,00142c59-745c-4004-a955-698ddcf1faa6,0.0,4.0,0.00000,7.0
3,0016f59f-9fb3-4ab5-ae78-9783314b81fc,0.0,10.0,0.00000,2.0
4,0017b9ef-5903-40f9-a219-85728eb78436,0.0,5.0,0.00000,5.0
...,...,...,...,...,...
14291,ffeaa17f-ee79-4f42-be58-afd60a5aef37,48.0,0.0,0.00000,NaN
14292,ffec4e11-c0a1-4fa4-8613-0979d6f46918,528.0,5.0,0.00947,100.0
14293,ffef31ea-cb4e-4d00-98ca-118df330cdca,0.0,2.0,0.00000,3.0
14294,ffef6048-9db0-4244-9750-3b530552f0ec,840.0,43.0,0.05119,1.0


In [82]:
temp_data1 = copy_data[copy_data['action']=='click_post'].groupby('source_event_id', as_index=False)['post_index_in_post_list'].agg('count')
temp_data1 = temp_data1.rename(columns={'post_index_in_post_list': 'count'})
temp_data2 = copy_data[copy_data['action']=='click_post'].groupby('source_event_id', as_index=False)['post_index_in_post_list'].agg('max')
temp_data = pd.merge(left=temp_data1, right=temp_data2, how="left")
temp_data['avg_index_dist'] = temp_data['post_index_in_post_list']/temp_data['count']
del temp_data['count']
del temp_data['post_index_in_post_list']
query_data = pd.merge(left = query_data, right = temp_data, how='left')
query_data

,source_event_id,numb_ads,numb_clicks,clicks_rate,first_click_index,avg_index_dist
0,00049ef3-c027-4f55-869c-46711f0ea0ad,24.0,0.0,0.00000,NaN,NaN
1,00069dda-4175-4bb0-8ef0-c0e1031d4da6,24.0,0.0,0.00000,NaN,NaN
2,00142c59-745c-4004-a955-698ddcf1faa6,0.0,4.0,0.00000,7.0,45.25000
3,0016f59f-9fb3-4ab5-ae78-9783314b81fc,0.0,10.0,0.00000,2.0,2.30000
4,0017b9ef-5903-40f9-a219-85728eb78436,0.0,5.0,0.00000,5.0,3.80000
...,...,...,...,...,...,...
14291,ffeaa17f-ee79-4f42-be58-afd60a5aef37,48.0,0.0,0.00000,NaN,NaN
14292,ffec4e11-c0a1-4fa4-8613-0979d6f46918,528.0,5.0,0.00947,100.0,101.00000
14293,ffef31ea-cb4e-4d00-98ca-118df330cdca,0.0,2.0,0.00000,3.0,32.50000
14294,ffef6048-9db0-4244-9750-3b530552f0ec,840.0,43.0,0.05119,1.0,19.27907


In [83]:
temp_data = data.groupby('source_event_id', as_index=False)['post_index_in_post_list'].agg(np.min)
condition = temp_data['post_index_in_post_list']<4
temp_data['first_three_ads'] = np.where(condition,1,0)
del temp_data['post_index_in_post_list']
query_data = pd.merge(left = query_data, right = temp_data, how='left')
query_data

,source_event_id,numb_ads,numb_clicks,clicks_rate,first_click_index,avg_index_dist,first_three_ads
0,00049ef3-c027-4f55-869c-46711f0ea0ad,24.0,0.0,0.00000,NaN,NaN,0
1,00069dda-4175-4bb0-8ef0-c0e1031d4da6,24.0,0.0,0.00000,NaN,NaN,0
2,00142c59-745c-4004-a955-698ddcf1faa6,0.0,4.0,0.00000,7.0,45.25000,0
3,0016f59f-9fb3-4ab5-ae78-9783314b81fc,0.0,10.0,0.00000,2.0,2.30000,1
4,0017b9ef-5903-40f9-a219-85728eb78436,0.0,5.0,0.00000,5.0,3.80000,0
...,...,...,...,...,...,...,...
14291,ffeaa17f-ee79-4f42-be58-afd60a5aef37,48.0,0.0,0.00000,NaN,NaN,0
14292,ffec4e11-c0a1-4fa4-8613-0979d6f46918,528.0,5.0,0.00947,100.0,101.00000,0
14293,ffef31ea-cb4e-4d00-98ca-118df330cdca,0.0,2.0,0.00000,3.0,32.50000,1
14294,ffef6048-9db0-4244-9750-3b530552f0ec,840.0,43.0,0.05119,1.0,19.27907,1


In [84]:
query_data.aggregate(np.mean)

numb_ads             56.213556
numb_clicks           5.301903
clicks_rate           0.062037
first_click_index    26.658595
avg_index_dist       15.366037
first_three_ads       0.354085
dtype: float64